In [ ]:
import csv
def extract_file(filename):
    
    # initializing rows list 
    rows = [] 

    # reading csv file 
    with open(filename, 'r') as csvfile: 

        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 


        # extracting each data row one by one 
        for row in csvreader: 
            rows.append(row) 

        # get total number of rows 
        print("Total no. of rows: %d"%(csvreader.line_num)) 
        
    return rows

In [ ]:
def Cleaning1(Data):
    New_data=list()
    for d in Data:
        New_data.append(d[1:])
    return New_data

In [ ]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
def remove_punctuation(string):
    punctuations='''!()[]{};:'"\,<>./?@#$%^&*_~'''
    for chars in string:
        if chars in punctuations:
            string=string.replace(chars,"")
    return(string)

def remove_alpha_numeric(string):
    line=list()
    for words in string.split(" "):
        if words.isalpha() == True and words not in stopwords:
            line.append(words)
    return(line)

def lower_case(string):
    S_list=list()
    for s in string:
        S_list.append(s.lower())     
    return(S_list)

def cleaning_text(data):
    cleaned_data=list()
    for d in data:
        d1=remove_punctuation(d[1])
        d1=remove_alpha_numeric(d1)
        d1=lower_case(d1)
        cleaned_data.append([int(d[0]),d1])
    return cleaned_data

In [ ]:
def create_vocubulary(data,threshold):
    Bag_of_words=list()
    for d in data.keys():
        if d not in Bag_of_words:
            if float(data[d])<=0.6 and float(data[d])>=0.3:
                Bag_of_words.append(d)
    return(Bag_of_words) 

In [ ]:
filename="train.csv\\train.csv"
Data=extract_file(filename)
print(Data[2])

In [ ]:
reqd_data=Cleaning1(Data[1:])
print(reqd_data[0])

In [ ]:
cleaned_data=cleaning_text(reqd_data)
print(cleaned_data[0])

In [ ]:
word_set=list()
for lab,dat in cleaned_data:
    word_set.append(dat)

In [ ]:
from tqdm import tqdm
import math  
def document_freq(words,document_list):
    freq=0
    for documents in document_list: 
        if words in documents:
            freq+=1
    return freq

def inverse_document_freq(document_list):
    corpus=dict()
    for documents in tqdm(document_list): 
        for words in documents:
            if words not in corpus.keys():
                DF=document_freq(words,document_list)
                IDF=math.log10(len(document_list))-math.log10(DF)
                corpus[words]=IDF
    return(corpus)


In [ ]:
logs=inverse_document_freq(word_set)

In [ ]:
word_bucket=list()
for keys in logs:
    if logs[keys]>=1.5 and logs[keys]<=3 and len(keys)>3 and keys not in stopwords:
        word_bucket.append(keys)

In [ ]:
len(word_bucket)

In [ ]:
#create dataset
def preparing_set(data,vocab):
    dataset=[]
    for lab,desc in data:
        vector=[]
        for d in vocab:
            if d in desc:
                vector.append(1)
            else:
                vector.append(0)
        dataset.append([vector,lab])
    return(dataset)

In [ ]:
Dataset=preparing_set(cleaned_data,word_bucket)

In [ ]:
import random
import numpy as np
random.shuffle(Dataset)

In [ ]:
X=[]
Y=[]
for data,lab in Dataset:
    X.append(data)
    Y.append(lab)
X=np.array(X).reshape(-1,len(word_bucket))

In [ ]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import seaborn as sns
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25)

In [ ]:

'''
#if you wanna use SVM
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, Y_train) 
svm_predictions = svm_model_linear.predict(X_test) 
accuracy = svm_model_linear.score(X_test, Y_test) 
print(accuracy)
cm = confusion_matrix(Y_test, svm_predictions) 
sns.heatmap(cm, annot=True)
plt.show()
import pickle
modelname='sentimentanalysis_svm'
with open (modelname,'wb') as file:
    pickle.dump(svm_model_linear,file)
'''

In [ ]:
#or use dense neural networ
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import numpy as np

model=Sequential()
model.add(Dense(32, activation='tanh', input_shape=(len(word_bucket),)))

model.add(Dense(256,activation='tanh'))
model.add(Dropout(0.2))

model.add(Dense(128,activation='tanh'))
model.add(Dropout(0.4))

model.add(Dense(64,activation='tanh'))

model.add(Dense(2,activation='sigmoid'))
adam=Adam(lr=0.00005)

model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
print(model.summary())


model.fit(X_train,Y_train,epochs=25,validation_data=(X_test,Y_test),batch_size=5,callbacks=None)

model.save("DL_sentimentanalysis_new.h5")

In [ ]:
pickle_out=open("word_bucket.pickle","wb")
pickle.dump(word_bucket,pickle_out)
pickle_out.close()